In [1]:
import numpy as np
import sys
import re
import os
import cygnus_lib as cy
import toolslib as tl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp
#import mylib as my
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line, resize, rescale)
import scipy.ndimage
from skimage.transform import hough_circle, hough_circle_peaks
from scipy.stats import norm
import scipy.stats as stats
from iDBSCAN import iDBSCAN


import pandas as pd
import pickle
import time
from IPython import display

%matplotlib inline
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
plt.rcParams['font.serif'] = ['Bitstream Vera Sans']
x_resolution = y_resolution = 2048
from itertools import combinations

In [2]:
t0 = time.time()

cy.set_atlas_style('large')
############# Data in imput #######################################
dataSelection = 'LAB'
#dataSelection = 'FNG'

runI = [494, 492, 493]
#runI = [492]
#runI = [813,814]

debug  = False
files = ""
################ analysis cards ################################
nsigma       = 1         # numero di sigma sopra il piedistallo
cimax        = 200       # valori del cut sull'imagine
iTr          = 0        # traccia di partenza
rescale      = 512       # binnagio finale immagine (deve essre un sottomultipli della 2**2 risluzione di partenza)
minClose     = 2         # minimum cluser size (rebinne image)
eps          = 5         # maximum distance for the cluster point
iterative    = 4         # number of iterations for the IDBSC
maxClose     = 30000     # massima dimesione del clustr evita le scriche      
Cmethod      = 'idbsc'    #'hdbs' # 'nccs' # 'dbsc' # 'idbsc'
max_image_to_read = 0  # 0 all
############### Inzializzazione varibili e costanti #################
scale        = int(x_resolution/rescale)
# MAIN LOOP ON 

for nRi in range(0,len(runI)):
    
    max_image=np.size(cy.swift_listdir(cy.file2FullPathCygnus(dataSelection, runI[nRi], 'H5')))
    print ("# of Image Files: %d" % (max_image))
    if max_image_to_read != 0:
        max_image = max_image_to_read   
    print ("WARNING: data will be anlyzed from %d to %d" % (iTr, max_image))

    if max_image == 0:
        print ("No file or image for file %s" % runI[nRi])
        print ("STOP")
        break

    #
    # load pedestal value generated by runs-pedestals.ipynb script
    # 
    try:
        fileoutm = ("./data/run%d_mean.h5" % (runI[nRi]))
        m_image = cy.read_image_h5(fileoutm)
        PedOverMax = m_image[m_image > cimax].size
        print ("Pedestal mean: %.2f, sigma: %.2f, over th. (%d) %d" % 
           (m_image[m_image<cimax].mean(), 
            np.sqrt(m_image[m_image<cimax].var()), cimax,
            (m_image>cimax).sum()))
    except:
        print ("No Pedestal file for run %s, run script runs-pedestals.ipynb" % runI[nRi])
        print ("STOP")
        break
    try: 
        fileouts = ("./data/run%d_sigma.h5" % (runI[nRi]))
        s_image = cy.read_image_h5(fileouts)
        print ("Sigma mean: %.2f, sigma: %.2f, over th. (50) %d" % 
       (s_image[s_image<50].mean(), 
        np.sqrt(s_image[s_image<50].var()), 
        (s_image>50).sum()))
    except:
        print ("No Sigma file for run %s, run script runs-pedestals.ipynb" % runI[nRi])
        print ("STOP")
        break
                
    #
    # Run by run init 
    #
    th_image   = np.round(m_image + nsigma*s_image) # verficare con il np.round.... np.ceil
    # th_image[:,:]=101 # per imostare tutto a 101
    data   = [] # output data 
    TrOk   = 0
    dCloseT= 0

    #################################
    # MAIN LOOP ON RUN
    ##################################
    
    for iTr in range(0, max_image):
        # load image
        image = cy.swift_read_image_h5(cy.imageFile2FullPathCygnus(dataSelection, runI[nRi], iTr)) 
        
        if (np.sum(image)/np.sum(m_image)) < 1.09: # Avoid wrong images 1.02 with we want to skip sparks too
            if image[image> cimax].size<1000000: # limit on the number og pixel over high cutof
                TrOk += 1
                print ("Analyzing Run %d Image %d \n" % (runI[nRi],iTr))
                print ("SumLight/SumPedestal = %.4f" % (np.sum(image)/np.sum(m_image)))
            
                edges_image     = (image > th_image) & (image < cimax)
                rebin_image     = cy.rebin(image, (rescale, rescale))
                rebin_th_image  = cy.rebin(th_image, (rescale, rescale))
                edges           = (rebin_image > rebin_th_image) & (rebin_image < cimax)         
                points          = np.array(np.nonzero(edges)).T.astype(float)
                
                clusters = iDBSCAN(iterative).fit(points)
                
                n_clusters_ = len(set(clusters.labels_)) - (1 if -1 in clusters.labels_ else 0)
                
                
                for Ci in range (0, n_clusters_): # number of clusters found in the image
                    cluvar   = [] # cluster info
                    #print ("Salving cluster %d from %d" % (Ci,n_clusters_))
                    Xi, Yi, Is, Ib, tag = tl.cluInfo(clusters,points,Ci,image,th_image,scale) # extract cluster information
                    # cluvar = [Run, Imag, Tag, X position, Y position, Light signal, Light pedestal]
                    cluvar.append(runI[nRi])
                    cluvar.append(iTr)
                    cluvar.append(tag)
                    cluvar.append(Xi)
                    cluvar.append(Yi)
                    cluvar.append(Is)
                    cluvar.append(Ib)
                    data.append(cluvar) # save the cluster info in a List
        print ("...Next image... \n")
t1 = time.time()
print ("Elapsed time of the algorithm: %.2f seconds / %.2f minutes" % ((t1-t0),(t1-t0)/60))

# of Image Files: 200
Pedestal mean: 99.33, sigma: 1.88, over th. (200) 1650
Sigma mean: 2.82, sigma: 2.16, over th. (50) 846
Analyzing Run 492 Image 0 

SumLight/SumPedestal = 1.0000
...Next image... 

Analyzing Run 492 Image 1 

SumLight/SumPedestal = 0.9986
...Next image... 

Analyzing Run 492 Image 2 

SumLight/SumPedestal = 1.0000
...Next image... 

Analyzing Run 492 Image 3 

SumLight/SumPedestal = 0.9991
...Next image... 

Analyzing Run 492 Image 4 

SumLight/SumPedestal = 1.0005
...Next image... 

Analyzing Run 492 Image 5 

SumLight/SumPedestal = 0.9991
...Next image... 

Analyzing Run 492 Image 6 

SumLight/SumPedestal = 0.9996
...Next image... 

Analyzing Run 492 Image 7 

SumLight/SumPedestal = 0.9988
...Next image... 

Analyzing Run 492 Image 8 

SumLight/SumPedestal = 0.9997
...Next image... 

Analyzing Run 492 Image 9 

SumLight/SumPedestal = 0.9991
...Next image... 

Analyzing Run 492 Image 10 

SumLight/SumPedestal = 1.0000
...Next image... 

Analyzing Run 492 Image 11

KeyboardInterrupt: 

In [ ]:
t2 = time.time()
#------------------- Saving File ------------------------------#
directory = ("./data/")          # Directory of the output file
filename  = ("ClustersTable_494-492-493")    # Name of the output file
extension = (".csv")             # Extension of the output file
dataout = directory + filename + extension # Full path of the output file

import pandas as pd

columns = ["Run","Image","Tag","X","Y","Light","Pedestal"]
df = pd.DataFrame(data, columns = columns)

#Now you have a csv with columns and index:
df.to_csv(dataout,index=False)

t3 = time.time()
print ("Table saving time: %.2f seconds / %.2f minutes" % ((t3-t2),(t3-t2)/60))